# Загрузка данных с агрегатора Banki.Ru

In [1]:
import time
import requests

import pandas as pd

from tqdm import tqdm

In [2]:
offset, limit = 0, 1000

df_banks_coords = []

while True:
    data = {
        'jsonrpc': '2.0',
        'method': 'bankGeo/getObjectsByFilter',
        'params': {
            'offset': offset,
            'limit': limit,
            'type': ['office', 'branch', 'atm', 'cash', 'self_office'],
            'bank_id': ['36119', '325', '63520', '327', '2764', '4389', '3697', '4725'],
        },
        'id': ''
    }
    
    r = requests.post('https://www.banki.ru/api/', json=data)
    if r.status_code != 200:
        raise Exception('Request has failed.')
    
    r = r.json()
    df_banks_coords.extend(r['result']['data'])
    
    if not len(df_banks_coords) < r['result']['total']:
        break
        
df_banks_coords = pd.DataFrame(df_banks_coords)

In [3]:
df_banks_coords.head()

,active,address,bank_id,icon_url,id,is_main,latitude,longitude,name,region_id,sort,type
0,True,"г. Казань, ул. Павлюхина, д. 101",36119,/upload/iblock/8e2/akbars.gif,10752354,True,55.765072,49.149404,Банкомат,479,0,atm
1,True,"Республика Татарстан, с. Молькеево, ул. Волков...",36119,/upload/iblock/8e2/akbars.gif,10754207,True,55.318403,47.815544,Банкомат,13615,0,atm
2,True,"г. Самара, ул. Спортивная, д. 5",36119,/upload/iblock/8e2/akbars.gif,10756829,True,53.187941,50.122472,Банкомат,489,0,atm
3,True,"г. Казань, ул. Вагапова, д. 3",36119,/upload/iblock/8e2/akbars.gif,10752496,True,55.783549,49.227291,Банкомат,479,0,atm
4,True,"г. Йошкар-Ола, просп. Царьградский, д. 37",36119,/upload/iblock/8e2/akbars.gif,10753962,True,56.637374,47.907160,Банкомат,394,0,atm


In [4]:
chunk_size = 100

df_banks_info = []

for i_start in tqdm(range(0, df_banks_coords.shape[0], chunk_size)):
    id_list = df_banks_coords.iloc[i_start:i_start+chunk_size]['id'].tolist()
    
    data = {
        'jsonrpc': '2.0',
        'method': 'bank/getBankObjectsData',
        'params': {
            'id_list': id_list
        },
        'id': '16'
    }
    
    r = requests.post('https://www.banki.ru/api/', json=data)
    if r.status_code != 200:
        raise Exception('Request has failed.')
    
    r = r.json()
    df_banks_info.extend(r['result']['data'])
    
    time.sleep(0.2)
    
df_banks_info = pd.DataFrame(df_banks_info)

100%|██████████| 280/280 [14:28<00:00,  3.10s/it]


In [5]:
df_banks_info.head()

,active,additional,address,bank_code,bank_icon_url,bank_id,bank_logo2_url,bank_logo_url,bank_name,bank_site,...,metro_name,name,phone,region_id,schedule_entities,schedule_general,schedule_private_person,schedule_vip,type,without_weekend
0,1,Валюта: рубли,"г. Казань, ул. Павлюхина, д. 101",akbars,/upload/iblock/8e2/akbars.gif,36119,https://www.banki.ru/upload/iblock/1a1/abb_135...,https://www.banki.ru/upload/iblock/68f/abb_100...,Ак Барс,www.akbars.ru,...,,Банкомат,,479,,,,,atm,1
1,1,Валюта: рубли,"Республика Татарстан, с. Молькеево, ул. Волков...",akbars,/upload/iblock/8e2/akbars.gif,36119,https://www.banki.ru/upload/iblock/1a1/abb_135...,https://www.banki.ru/upload/iblock/68f/abb_100...,Ак Барс,www.akbars.ru,...,,Банкомат,,13615,,,,,atm,1
2,1,Валюта: рубли,"г. Самара, ул. Спортивная, д. 5",akbars,/upload/iblock/8e2/akbars.gif,36119,https://www.banki.ru/upload/iblock/1a1/abb_135...,https://www.banki.ru/upload/iblock/68f/abb_100...,Ак Барс,www.akbars.ru,...,,Банкомат,,489,,,,,atm,1
3,1,Валюта: рубли,"г. Казань, ул. Вагапова, д. 3",akbars,/upload/iblock/8e2/akbars.gif,36119,https://www.banki.ru/upload/iblock/1a1/abb_135...,https://www.banki.ru/upload/iblock/68f/abb_100...,Ак Барс,www.akbars.ru,...,,Банкомат,,479,,,,,atm,1
4,1,Валюта: рубли,"г. Йошкар-Ола, просп. Царьградский, д. 37",akbars,/upload/iblock/8e2/akbars.gif,36119,https://www.banki.ru/upload/iblock/1a1/abb_135...,https://www.banki.ru/upload/iblock/68f/abb_100...,Ак Барс,www.akbars.ru,...,,Банкомат,,394,,,,,atm,1


In [6]:
df_banks_info.columns

Index(['active', 'additional', 'address', 'bank_code', 'bank_icon_url',
       'bank_id', 'bank_logo2_url', 'bank_logo_url', 'bank_name', 'bank_site',
       'comment_to_address', 'id', 'is_at_closed_place', 'is_main_office',
       'is_round_the_clock', 'is_works_as_shop', 'latitude', 'longitude',
       'metro_name', 'name', 'phone', 'region_id', 'schedule_entities',
       'schedule_general', 'schedule_private_person', 'schedule_vip', 'type',
       'without_weekend'],
      dtype='object')

In [8]:
df_banks_info = df_banks_info[['id', 'bank_name', 'name', 'type', 'address', 'comment_to_address',
                               'latitude', 'longitude', 'is_round_the_clock']]

df_banks_info.rename(columns={
    'latitude': 'lat',
    'longitude': 'lng',
    'comment_to_address': 'comment',
    'bank_name': 'bank',
    'is_round_the_clock': 'access24h'
}, inplace=True)

df_banks_info.head()

,id,bank,name,type,address,comment,lat,lng,access24h
0,10752354,Ак Барс,Банкомат,atm,"г. Казань, ул. Павлюхина, д. 101",Супермаркет &laquo;Эдельвейс&raquo;,55.765072,49.149404,False
1,10754207,Ак Барс,Банкомат,atm,"Республика Татарстан, с. Молькеево, ул. Волков...",Магазин &laquo;Затлы&raquo;,55.318403,47.815544,False
2,10756829,Ак Барс,Банкомат,atm,"г. Самара, ул. Спортивная, д. 5",ТК &laquo;Журавль&raquo;,53.187941,50.122472,True
3,10752496,Ак Барс,Банкомат,atm,"г. Казань, ул. Вагапова, д. 3",ТК &laquo;Долина изобилия&raquo;,55.783549,49.227291,False
4,10753962,Ак Барс,Банкомат,atm,"г. Йошкар-Ола, просп. Царьградский, д. 37",Супермаркет &laquo;Eurospar&raquo;,56.637374,47.90716,True


In [9]:
df_banks_info.to_csv('base_bankiru.csv', sep=',', index=False)

# Загрузка данных с сайта Росбанка

In [1]:
import requests
from bs4 import BeautifulSoup

import urllib

import re
from operator import attrgetter, itemgetter

## Скачиваем страницы со списком банкоматов Росбанка

In [65]:
# кусок HTML-кода выдран из меню выбора региона на сайте Росбанка

html = """
<div class="container">
            <div class="row">
<div class="col-md-3 col-sm-3 col-xs-12"><div class="popup-city__body_section">
            <div class="popup-city__body_title">А</div>
            <ul class="popup-city__body_list"><li><a href="?region=22" title="Алтайский край">Алтайский край</a></li><li><a href="?region=28" title="Амурская область">Амурская область</a></li><li><a href="?region=29" title="Архангельская область">Архангельская область</a></li><li><a href="?region=30" title="Астраханская область">Астраханская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Б</div>
            <ul class="popup-city__body_list"><li><a href="?region=31" title="Белгородская область">Белгородская область</a></li><li><a href="?region=32" title="Брянская область">Брянская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">В</div>
            <ul class="popup-city__body_list"><li><a href="?region=33" title="Владимирская область">Владимирская область</a></li><li><a href="?region=34" title="Волгоградская область">Волгоградская область</a></li><li><a href="?region=35" title="Вологодская область">Вологодская область</a></li><li><a href="?region=36" title="Воронежская область">Воронежская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Е</div>
            <ul class="popup-city__body_list"><li><a href="?region=79" title="Еврейская автономная область">Еврейская автономная область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">З</div>
            <ul class="popup-city__body_list"><li><a href="?region=75" title="Забайкальский край">Забайкальский край</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">И</div>
            <ul class="popup-city__body_list"><li><a href="?region=37" title="Ивановская область">Ивановская область</a></li><li><a href="?region=38" title="Иркутская область">Иркутская область</a></li></ul>
            </div></div>
                <div class="col-md-3 col-sm-3 col-xs-12"><div class="popup-city__body_section">
            <div class="popup-city__body_title">К</div>
            <ul class="popup-city__body_list"><li><a href="?region=39" title="Калининградская область">Калининградская область</a></li><li><a href="?region=40" title="Калужская область">Калужская область</a></li><li><a href="?region=41" title="Камчатский край">Камчатский край</a></li><li><a href="?region=42" title="Кемеровская область">Кемеровская область</a></li><li><a href="?region=43" title="Кировская область">Кировская область</a></li><li><a href="?region=44" title="Костромская область">Костромская область</a></li><li><a href="?region=23" title="Краснодарский край">Краснодарский край</a></li><li><a href="?region=24" title="Красноярский край">Красноярский край</a></li><li><a href="?region=45" title="Курганская область">Курганская область</a></li><li><a href="?region=46" title="Курская область">Курская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Л</div>
            <ul class="popup-city__body_list"><li><a href="?region=48" title="Липецкая область">Липецкая область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">М</div>
            <ul class="popup-city__body_list"><li><a href="?region=49" title="Магаданская область">Магаданская область</a></li><li><a href="?region=77" title="Москва и Московская область">Москва и Московская область</a></li><li><a href="?region=51" title="Мурманская область">Мурманская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Н</div>
            <ul class="popup-city__body_list"><li><a href="?region=52" title="Нижегородская область">Нижегородская область</a></li><li><a href="?region=53" title="Новгородская область">Новгородская область</a></li><li><a href="?region=54" title="Новосибирская область">Новосибирская область</a></li></ul>
            </div></div>
                <div class="col-md-3 col-sm-3 col-xs-12"><div class="popup-city__body_section">
            <div class="popup-city__body_title">О</div>
            <ul class="popup-city__body_list"><li><a href="?region=55" title="Омская область">Омская область</a></li><li><a href="?region=56" title="Оренбургская область">Оренбургская область</a></li><li><a href="?region=57" title="Орловская область">Орловская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">П</div>
            <ul class="popup-city__body_list"><li><a href="?region=58" title="Пензенская область">Пензенская область</a></li><li><a href="?region=59" title="Пермский край">Пермский край</a></li><li><a href="?region=25" title="Приморский край">Приморский край</a></li><li><a href="?region=60" title="Псковская область">Псковская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Р</div>
            <ul class="popup-city__body_list"><li><a href="?region=1" title="Республика Адыгея">Республика Адыгея</a></li><li><a href="?region=2" title="Республика Башкортостан">Республика Башкортостан</a></li><li><a href="?region=3" title="Республика Бурятия">Республика Бурятия</a></li><li><a href="?region=8" title="Республика Калмыкия">Республика Калмыкия</a></li><li><a href="?region=10" title="Республика Карелия">Республика Карелия</a></li><li><a href="?region=11" title="Республика Коми">Республика Коми</a></li><li><a href="?region=12" title="Республика Марий Эл">Республика Марий Эл</a></li><li><a href="?region=13" title="Республика Мордовия">Республика Мордовия</a></li><li><a href="?region=14" title="Республика Саха (Якутия)">Республика Саха (Якутия)</a></li><li><a href="?region=16" title="Республика Татарстан">Республика Татарстан</a></li><li><a href="?region=17" title="Республика Тыва">Республика Тыва</a></li><li><a href="?region=19" title="Республика Хакасия">Республика Хакасия</a></li><li><a href="?region=61" title="Ростовская область">Ростовская область</a></li><li><a href="?region=62" title="Рязанская область">Рязанская область</a></li></ul>
            </div></div>
                <div class="col-md-3 col-sm-3 col-xs-12"><div class="popup-city__body_section">
            <div class="popup-city__body_title">С</div>
            <ul class="popup-city__body_list"><li><a href="?region=63" title="Самарская область">Самарская область</a></li><li><a href="?region=78" title="Санкт-Петербург и Ленинградская область">Санкт-Петербург и Ленинградская область</a></li><li><a href="?region=64" title="Саратовская область">Саратовская область</a></li><li><a href="?region=65" title="Сахалинская область">Сахалинская область</a></li><li><a href="?region=66" title="Свердловская область">Свердловская область</a></li><li><a href="?region=67" title="Смоленская область">Смоленская область</a></li><li><a href="?region=26" title="Ставропольский край">Ставропольский край</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Т</div>
            <ul class="popup-city__body_list"><li><a href="?region=68" title="Тамбовская область">Тамбовская область</a></li><li><a href="?region=69" title="Тверская область">Тверская область</a></li><li><a href="?region=70" title="Томская область">Томская область</a></li><li><a href="?region=71" title="Тульская область">Тульская область</a></li><li><a href="?region=72" title="Тюменская область">Тюменская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">У</div>
            <ul class="popup-city__body_list"><li><a href="?region=18" title="Удмуртская Республика">Удмуртская Республика</a></li><li><a href="?region=73" title="Ульяновская область">Ульяновская область</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Х</div>
            <ul class="popup-city__body_list"><li><a href="?region=27" title="Хабаровский край">Хабаровский край</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Ч</div>
            <ul class="popup-city__body_list"><li><a href="?region=74" title="Челябинская область">Челябинская область</a></li><li><a href="?region=21" title="Чувашская Республика">Чувашская Республика</a></li></ul>
            </div><div class="popup-city__body_section">
            <div class="popup-city__body_title">Я</div>
            <ul class="popup-city__body_list"><li><a href="?region=76" title="Ярославская область">Ярославская область</a></li></ul>
            </div></div>            </div>
        </div>
"""

In [66]:
soup = BeautifulSoup(html, 'html.parser')
regions = map(itemgetter('href'), soup.find_all('a', href=True))
regions = map(lambda s: re.search('\d+', s).group(0), regions)
regions = list(regions)

In [67]:
regions_names = map(itemgetter('title'), soup.find_all('a', href=True))
regions_names = list(regions_names)

In [4]:
URL_FORMAT = 'https://www.rosbank.ru/ru/atms/list.php?p_f_2_11={city}&metrocity=-1&p_f_2_22=0&street=&p_f_2_temp_id=459&p_f_2_all=1&page_25={page}'

In [5]:
!mkdir -p pages

In [8]:
def process_page(sess, region, city, page):
    url = URL_FORMAT.format(city=urllib.parse.quote(city), page=page + 1)
    print(url)
    r = sess.get(url)
    
    city = re.sub('\s', '_', city)
    with open(f'pages/{region:02d}_{city}_{page:03d}.html', 'w') as f_html:
        f_html.write(r.text)
        
    return r

def process_region(sess, region, cities):
    for city in cities:
        print(city)
        
#         if city[0].islower():
#             continue
    
        r = process_page(sess, region, city, 0)
        if r.status_code != 200:
            print("FAIL", r.status_code)
    
        try:
            soup = BeautifulSoup(r.text, 'html.parser')
            pages = soup.find('ul', class_='pagination__inner')
            pages = pages.find_all('a', href=True, class_='pagination-page')
            pages = max(map(int,map(attrgetter('text'), pages)))
        except:
            pages = 1
    
        for page in range(1, pages):
            r = process_page(sess, region, city, page)
            if r.status_code != 200:
                print("FAIL", r.status_code)

In [ ]:
for region in regions:
    with requests.Session() as sess:
        sess.cookies.update({ 'regionrb': region })
        r = sess.get('https://www.rosbank.ru/ru/atms/')
        
        soup = BeautifulSoup(r.text, 'html.parser')
        
        cities = soup.find('select', class_='city_select')
        cities = cities.find_all('option')
        cities = map(attrgetter('text'), cities)
        cities = list(cities)
        
        print(region, len(cities), r.status_code)
        
        process_region(sess, int(region), cities)

## Скачиваем карты c местоположением банкоматов

In [2]:
import os

In [7]:
from urllib.parse import parse_qsl, urlparse

def extract_atm_info_file(filename):
    atm_info_list = []
    
    with open(filename, mode='r') as f_name:
        soup = BeautifulSoup(f_name, 'html.parser')
        
    _, name = os.path.split(filename)
    name, _ = os.path.splitext(name)
    parts = name.split('_')
    
    region_rb = parts[0]
    city = ' '.join(parts[1:-1])

    atms = soup.find_all('div', class_="page-atm__table_row")
    for atm in atms:
        atm_info = {
            'bank': atm.find('div', class_="address-logo").text.strip(),
            'address': atm.find('div', class_="address-title").text.strip(),
            'name': atm.find('div', class_="address-type").text.strip(),
            'access24h': 'круглосуточно' in atm.find('div', class_="page-atm__table_col--time").text.lower(),
            'id': atm.find('div', class_="address-map").find('a', href=True)["href"],
            'region_rb': int(region_rb),
            'city': city,
        }
        
        atm_info['id'] = dict(parse_qsl(urlparse(atm_info['id']).query))['showonmap']
    
        metro = atm.find('div', class_="metro-station")
        if metro is not None:
            atm_info['metro'] = re.sub('\s+', ' ', metro.text.strip())
            
        atm_info_list.append(atm_info)
        
    return atm_info_list

In [ ]:
atm_info_list = []

for root, dirs, filenames in os.walk('pages'):
    for filename in sorted(filenames):
        filename = os.path.join(root, filename)
        try:
            atm_info_list.extend(extract_atm_info_file(filename))
        except Exception as e:
            print(filename)
            raise e
        
atm_info_list[:5]     

In [5]:
atm_urls = map(itemgetter('id'), atm_info_list)
atm_urls = map(lambda s: 'https://www.rosbank.ru/ru/atms/?showonmap={}'.format(s), atm_urls)

region_urls = map(itemgetter('region_rb'), atm_info_list)

atm_urls = list(set(zip(atm_urls, region_urls)))
atm_urls[:5]

[('https://www.rosbank.ru/ru/atms/?showonmap=62584&test=1', 77),
 ('https://www.rosbank.ru/ru/atms/?showonmap=80131', 38),
 ('https://www.rosbank.ru/ru/atms/?showonmap=75389&test=1', 77),
 ('https://www.rosbank.ru/ru/atms/?showonmap=69881&test=1', 77),
 ('https://www.rosbank.ru/ru/atms/?showonmap=82021&test=1', 77)]

In [6]:
len(atm_urls)

10194

In [8]:
!mkdir -p atms

In [9]:
def process_atm_page(region, url):    
    with requests.Session() as sess:
        sess.cookies.update({ 'regionrb': str(region) })
        r = sess.get(url)
        
    with open(f'atms/{region:02d}_{atm_id}.html', 'w') as f_html:
        f_html.write(r.text)
        
    return r

def process_atm_page_wrapper(pair):
    return process_atm_page(*pair[::-1])

In [10]:
from multiprocessing.pool import ThreadPool

In [11]:
%%time

pool = ThreadPool(8)
r = pool.map(process_atm_page_wrapper, atm_urls)

CPU times: user 4min 30s, sys: 32.1 s, total: 5min 2s
Wall time: 32min 29s


In [12]:
all(map(lambda k: k.status_code == 200, r))

True

## Формируем датасет

In [2]:
import pandas as pd
import numpy as np

In [9]:
%%time

df_atm_info = []

for root, dirs, filenames in os.walk('pages'):
    for filename in sorted(filenames):
        filename = os.path.join(root, filename)
        df_atm_info.extend(extract_atm_info_file(filename))
        
df_atm_info[:5]

CPU times: user 2min 59s, sys: 301 ms, total: 2min 59s
Wall time: 3min 10s


In [14]:
df_atm_info = pd.DataFrame(df_atm_info)
df_atm_info.head()

,access24h,address,bank,city,id,metro,name,region_rb
0,False,Улица Партизана Железняка 35А,Росбанк,Красноярск,45351,NaN,Бизнес Центр Сириус ХАБ ПАО РОСБАНК,1
1,False,Улица Тургеневское шоссе 27,Росбанк,Новая Адыгея,44392,NaN,"магазин ""Декатлон""",1
2,False,"Тургеневское ш., 27",Альфабанк,Новая Адыгея,81782,NaN,,1
3,False,"поселок Новая Адыгея, шоссе Тургеневское, дом 27",УРАЛСИБ Банк,Новая Адыгея,61293,NaN,"Входная группа ТЦ ""ИКЕА""",1
4,True,"Адыгейская ул., 93",Альфабанк,Тахтамукай,83137,NaN,,1


In [11]:
COORD_PATTERN = re.compile('var long=(\d+\.\d+); var lati=(\d+\.\d+);')

def extract_atm_coords(filename):
    with open(filename, mode='r') as f_atm:
        soup = BeautifulSoup(f_atm, 'html.parser')
    
    _, name = os.path.split(filename)
    name, _ = os.path.splitext(name)
    name = name.split('_')[1]
    
    script = soup.find('div', class_='page-atm__map').find('script').text
    script = map(str.strip, script.split('\n'))
    
    lng, lat = None, None
    
    for s in script:
        if s.startswith('var'):
            res = COORD_PATTERN.search(s)
            if res is not None:
                lng, lat = res.group(1), res.group(2)
                break
    
    return {'id': name, 'lng': lng, 'lat': lat}

In [17]:
%%time

df_atm_coords = []

for root, dirs, filenames in os.walk('atms'):
    for filename in sorted(filenames):
        filename = os.path.join(root, filename)
        try:
            df_atm_coords.append(extract_atm_coords(filename))
        except Exception as e:
            print(filename)
        
df_atm_coords[:5]

atms/03_46063.html
atms/17_44107.html
atms/17_45284.html
atms/25_44667.html
atms/25_45485.html
atms/25_45578.html
atms/25_82239.html
atms/27_50041.html
atms/27_50050.html
atms/30_44668.html
atms/37_44626.html
atms/37_82535.html
atms/49_75072.html
atms/51_45355.html
atms/52_45087.html
atms/55_79041.html
atms/72_44801.html
atms/75_45721.html
atms/77_62931.html
CPU times: user 10min 50s, sys: 1.07 s, total: 10min 51s
Wall time: 11min 54s


In [18]:
df_atm_coords = pd.DataFrame(df_atm_coords)
df_atm_coords.head()

,id,lat,lng
0,44392,45.013084,38.929328
1,45351,56.035552,92.930698
2,61293,45.013084,38.929328
3,75759,51.810559,107.607134
4,81782,45.012358,38.928279


In [166]:
df_atm = pd.merge(df_atm_info, df_atm_coords, on='id')
df_atm.dropna(subset=['lat', 'lng'], inplace=True)
df_atm.drop_duplicates(subset=['id'], inplace=True)
df_atm.drop_duplicates(subset=['lat', 'lng'], inplace=True)
df_atm.head(10)

,access24h,address,bank,city,id,metro,name,region_rb,lat,lng
0,False,Улица Партизана Железняка 35А,Росбанк,Красноярск,45351,NaN,Бизнес Центр Сириус ХАБ ПАО РОСБАНК,1,56.035552,92.930698
1,False,Улица Тургеневское шоссе 27,Росбанк,Новая Адыгея,44392,NaN,"магазин ""Декатлон""",1,45.013084,38.929328
2,False,"Тургеневское ш., 27",Альфабанк,Новая Адыгея,81782,NaN,,1,45.012358,38.928279
5,True,"Адыгейская ул., 93",Альфабанк,Тахтамукай,83137,NaN,,1,44.9290606,38.9800929
6,False,"ул. Геологическая, 28",Газпромбанк,Улан-Удэ,75759,NaN,"САО ""ВСК""",1,51.810559,107.607134
7,False,"ул. Пушкина, 27",Альфабанк,Белорецк,82196,NaN,,2,53.971524,58.409304
8,False,город Белорецк,УРАЛСИБ Банк,Белорецк,81737,NaN,,2,53.967621,58.410023
10,False,"город Белорецк, улица Кирова, дом 48",УРАЛСИБ Банк,Белорецк,62000,NaN,,2,53.962207,58.401435
12,False,"город Белорецк, улица Точисского, дом 1, строе...",УРАЛСИБ Банк,Белорецк,61965,NaN,,2,53.971667,58.402181
20,True,"город Белорецк, улица Пушкина, дом 27",УРАЛСИБ Банк,Белорецк,61769,NaN,,2,53.97154,58.409286


In [167]:
def preprocess_cities(city):
    city = re.sub('\(.*?\)', '', city)
    city = re.sub('\S+\.', '', city)
    city = re.sub(',(?:.*?)', '', city)
    city = re.sub('поселок городского типа', '', city)
    city = re.sub('(?:город|cтаница|поселок|село|ст\-ца|деревня)', '', city)
    city = city.strip().title()
    
    if city == 'Лесной Ок':
        city = 'Лесной Городок'
    elif city == 'Ского Типа Белоозерский':
        city = 'Белоозерский'
    elif city == 'Совхоз Им Ленина':
        city = 'поселок совхоза имени Ленина'
    elif city == 'Новая Тура С':
        city = 'Новая Тура'
    elif city == 'Нижний Нов':
        city = 'Нижний Новгород'
    elif city == 'Великий Нов':
        city = 'Великий Новгород'
    elif city == 'Бурнашево С':
        city = 'Бурнашево'
    elif city == 'Апастово Пгт':
        city = 'Апастово'
    return city

In [168]:
df_atm['city'] = df_atm['city'].apply(preprocess_cities)

In [170]:
df_atm['city'].unique().shape

(887,)

In [171]:
regions_map = dict(zip(map(int, regions), regions_names))
df_atm['region_name'] = df_atm['region_rb'].map(regions_map)

banks_map = {
    'Росбанк': 'Росбанк',
    'Альфабанк': 'Альфа-Банк',
    'УРАЛСИБ Банк': 'Банк Уралсиб',
    'Газпромбанк': 'Газпромбанк',
    'АК БАРС': 'Ак Барс',
    'Россельхозбанк': 'Россельхозбанк',
    'Райффайзен-банк': 'Райффайзенбанк'
}
df_atm['bank'] = df_atm['bank'].map(banks_map)

df_atm.head(10)

,access24h,address,bank,city,id,metro,name,region_rb,lat,lng,region_name
0,False,Улица Партизана Железняка 35А,Росбанк,Красноярск,45351,NaN,Бизнес Центр Сириус ХАБ ПАО РОСБАНК,1,56.035552,92.930698,Республика Адыгея
1,False,Улица Тургеневское шоссе 27,Росбанк,Новая Адыгея,44392,NaN,"магазин ""Декатлон""",1,45.013084,38.929328,Республика Адыгея
2,False,"Тургеневское ш., 27",Альфа-Банк,Новая Адыгея,81782,NaN,,1,45.012358,38.928279,Республика Адыгея
5,True,"Адыгейская ул., 93",Альфа-Банк,Тахтамукай,83137,NaN,,1,44.9290606,38.9800929,Республика Адыгея
6,False,"ул. Геологическая, 28",Газпромбанк,Улан-Удэ,75759,NaN,"САО ""ВСК""",1,51.810559,107.607134,Республика Адыгея
7,False,"ул. Пушкина, 27",Альфа-Банк,Белорецк,82196,NaN,,2,53.971524,58.409304,Республика Башкортостан
8,False,город Белорецк,Банк Уралсиб,Белорецк,81737,NaN,,2,53.967621,58.410023,Республика Башкортостан
10,False,"город Белорецк, улица Кирова, дом 48",Банк Уралсиб,Белорецк,62000,NaN,,2,53.962207,58.401435,Республика Башкортостан
12,False,"город Белорецк, улица Точисского, дом 1, строе...",Банк Уралсиб,Белорецк,61965,NaN,,2,53.971667,58.402181,Республика Башкортостан
20,True,"город Белорецк, улица Пушкина, дом 27",Банк Уралсиб,Белорецк,61769,NaN,,2,53.97154,58.409286,Республика Башкортостан


In [172]:
df_atm.shape

(7903, 11)

In [174]:
df_atm.to_csv('dataset_rosbank.csv', sep=',', index=False)

## Приводим адреса банкоматов к единому виду

In [1]:
import requests

In [2]:
from functools import wraps

import time

def timeout(rps=5):
    t_max = 1.0 / rps

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            t_start = time.time()
            res = func(*args, **kwargs)
            t_delta = time.time() - t_start

            if t_delta < t_max:
                time.sleep(t_max - t_delta)
            return res
        return wrapper
    return decorator

In [3]:
with open('yandex-api.key', mode='r') as f_key:
    API_KEY = next(f_key).strip()

In [4]:
def yandex_request_coord(lat, lon):
    r = requests.get("https://geocode-maps.yandex.ru/1.x/",
                     params={"format": "json",
                             "apikey": API_KEY,
                             "geocode": f"{lat},{lon}",
                             "results": 1})
    return r

def yandex_request_address(address):
    r = requests.get("https://geocode-maps.yandex.ru/1.x/",
                     params={"format": "json",
                             "apikey": API_KEY,
                             "geocode": f"{address}",
                             "results": 1})
    return r

@timeout(rps=5)
def yandex_request(args):
    if len(args) == 2:
        return yandex_request_coord(*args)
    elif len(args) == 1:
        return yandex_request_address(*args)
    else:
        raise TypeError('yandex_request(args) takes 1 positional argument. 1 <= len(args) <= 2')

In [5]:
from tqdm import tqdm

In [14]:
import pandas as pd

df_atm = pd.read_csv('dataset_rosbank.csv', sep=',')
df_atm['address'] = df_atm['address'].fillna('')
df_atm.head()

,access24h,address,bank,city,id,metro,name,region_rb,lat,lng,region_name
0,False,Улица Партизана Железняка 35А,Росбанк,Красноярск,45351,NaN,Бизнес Центр Сириус ХАБ ПАО РОСБАНК,1,56.035552,92.930698,Республика Адыгея
1,False,Улица Тургеневское шоссе 27,Росбанк,Новая Адыгея,44392,NaN,"магазин ""Декатлон""",1,45.013084,38.929328,Республика Адыгея
2,False,"Тургеневское ш., 27",Альфа-Банк,Новая Адыгея,81782,NaN,NaN,1,45.012358,38.928279,Республика Адыгея
3,True,"Адыгейская ул., 93",Альфа-Банк,Тахтамукай,83137,NaN,NaN,1,44.929061,38.980093,Республика Адыгея
4,False,"ул. Геологическая, 28",Газпромбанк,Улан-Удэ,75759,NaN,"САО ""ВСК""",1,51.810559,107.607134,Республика Адыгея


In [7]:
df_atm[['lat', 'lng']].isna().any()

lat    False
lng    False
dtype: bool

In [ ]:
r_yandex = {}

for r in tqdm(df_atm.itertuples()):
    args = (r.lng, r.lat)

    if r.id in r_yandex:
        continue
    
    res = yandex_request(args)
    if res.status_code != 200:
        print("FAILED: ", args)
    r_yandex[r.id] = res

In [11]:
import pickle

with open('yandex-rosbank.dump', mode='wb') as f_dump:
    pickle.dump(r_yandex, f_dump)

In [ ]:
import pickle

with open('yandex-rosbank.dump', mode='rb') as f_dump:
    r_yandex = pickle.load(f_dump)

In [12]:
df_atms_address = []

for id_, r in r_yandex.items():
    atm_info = {'id': id_}
    
    r = r_yandex[id_].json()['response']['GeoObjectCollection']['featureMember']
    if len(r):
        r = r[0]['GeoObject']
        
        try:
            address = r['metaDataProperty']['GeocoderMetaData']['Address']['formatted']
        except:
            address = ''
        finally:
            atm_info['address'] = address
        
    df_atms_address.append(atm_info)
    
df_atms_address = pd.DataFrame(df_atms_address)
df_atms_address.head()

,address,id
0,"Красноярск, улица Партизана Железняка, 35А",45351
1,"Республика Адыгея, Тахтамукайский район, посёл...",44392
2,"Республика Адыгея, Тахтамукайский район, посёл...",81782
3,"Республика Адыгея, Тахтамукайский район, аул Т...",83137
4,"Республика Бурятия, Улан-Удэ, Геологическая ул...",75759


In [15]:
df_atm.drop(columns=['address'], inplace=True)
df_atm = pd.merge(df_atm, df_atms_address, on='id')
df_atm.head()

,access24h,bank,city,id,metro,name,region_rb,lat,lng,region_name,address
0,False,Росбанк,Красноярск,45351,NaN,Бизнес Центр Сириус ХАБ ПАО РОСБАНК,1,56.035552,92.930698,Республика Адыгея,"Красноярск, улица Партизана Железняка, 35А"
1,False,Росбанк,Новая Адыгея,44392,NaN,"магазин ""Декатлон""",1,45.013084,38.929328,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, посёл..."
2,False,Альфа-Банк,Новая Адыгея,81782,NaN,NaN,1,45.012358,38.928279,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, посёл..."
3,True,Альфа-Банк,Тахтамукай,83137,NaN,NaN,1,44.929061,38.980093,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, аул Т..."
4,False,Газпромбанк,Улан-Удэ,75759,NaN,"САО ""ВСК""",1,51.810559,107.607134,Республика Адыгея,"Республика Бурятия, Улан-Удэ, Геологическая ул..."


In [16]:
df_atm.to_csv('dataset_rosbank.csv', sep=',', index=False)

## Заполняем пустое поле name в дампе Росбанка

In [52]:
df_atm = pd.read_csv('dataset_rosbank.csv', sep=',')
df_atm['address'] = df_atm['address'].fillna('')
df_atm.head()

,access24h,bank,city,id,metro,name,region_rb,lat,lng,region_name,address
0,False,Росбанк,Красноярск,45351,NaN,Бизнес Центр Сириус ХАБ ПАО РОСБАНК,1,56.035552,92.930698,Республика Адыгея,"Красноярск, улица Партизана Железняка, 35А"
1,False,Росбанк,Новая Адыгея,44392,NaN,"магазин ""Декатлон""",1,45.013084,38.929328,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, посёл..."
2,False,Альфа-Банк,Новая Адыгея,81782,NaN,NaN,1,45.012358,38.928279,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, посёл..."
3,True,Альфа-Банк,Тахтамукай,83137,NaN,NaN,1,44.929061,38.980093,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, аул Т..."
4,False,Газпромбанк,Улан-Удэ,75759,NaN,"САО ""ВСК""",1,51.810559,107.607134,Республика Адыгея,"Республика Бурятия, Улан-Удэ, Геологическая ул..."


In [53]:
df_atm.isna().any(axis=0)

access24h      False
bank           False
city           False
id             False
metro           True
name            True
region_rb      False
lat            False
lng            False
region_name    False
address        False
dtype: bool

In [54]:
df_banki = pd.read_csv('./base_bankiru.csv')
df_banki = df_banki[df_banki["type"] == "atm"]
df_banki.dropna(subset=['lat', 'lon'], inplace=True)
df_banki.head()

,bank,name,type,address,comment,lat,lon,access24h
0,Альфа-Банк,Банкомат,atm,"Алтайский край, с. Алтайское",Алтай Пэлас,51.821321,85.665832,True
1,ВТБ,Банкомат,atm,"Алтайский край, с. Алтайское, ул. Советское, д...",Магазин «Мария-Ра»,51.949234,85.337500,False
4,Ак Барс,Банкомат,atm,"г. Барнаул, ул. Попова, д. 51",ОО «Барнаульский № 2»,53.367107,83.677421,True
5,Ак Барс,Банкоматы,atm,"г. Барнаул, просп. Красноармейский, д. 75 Б",ОО «Барнаульский № 1»,53.339939,83.771164,True
6,Альфа-Банк,Банкомат,atm,"г. Барнаул, Красноармейский пр-кт, 47А",ТРК «Сити-центр»,53.335253,83.775410,False


In [55]:
df_banki.shape

(17791, 8)

In [56]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import DistanceMetric

import numpy as np

In [57]:
from tqdm import tqdm

In [58]:
bank_index = []
bank_dists = []

R = 6373.0

metric = DistanceMetric.get_metric('haversine')

b = np.deg2rad(df_banki[["lat", "lon"]]).values.reshape(-1, 2)
a = np.deg2rad(df_atm[["lat", "lng"]]).values.reshape(-1, 2)

for i in tqdm(range(df_atm.shape[0])):
    dists = metric.pairwise(a[i].reshape(-1, 2), b) * R
    j = np.argsort(dists, axis=1)[0,0]
    d = dists[0,j]
    
    if not d < 0.125:
        j = None       
        
    bank_index.append(j)
    bank_dists.append(d)

100%|██████████| 7903/7903 [00:24<00:00, 325.16it/s]


In [59]:
sum(map(lambda s: s is None, bank_index))

1759

In [60]:
for r in df_atm.itertuples():
    if r.bank == 'Росбанк':
        continue
    
    i = bank_index[r.Index]
    if i is None:
        continue
    
    df_atm['name'].at[r.Index] = df_banki["comment"].iloc[i]

In [61]:
df_atm['name'] = df_atm['name'].fillna('')

mask = df_atm['name'] == ''
print(mask.sum())
df_atm[mask].head(10)

1318


,access24h,bank,city,id,metro,name,region_rb,lat,lng,region_name,address
3,True,Альфа-Банк,Тахтамукай,83137,NaN,,1,44.929061,38.980093,Республика Адыгея,"Республика Адыгея, Тахтамукайский район, аул Т..."
8,False,Банк Уралсиб,Белорецк,61965,NaN,,2,53.971667,58.402181,Республика Башкортостан,"Республика Башкортостан, Белорецк, улица П. То..."
16,False,Банк Уралсиб,Белорецк,61317,NaN,,2,53.959877,58.366401,Республика Башкортостан,"Республика Башкортостан, Белорецк, улица В. Бл..."
22,False,Ак Барс,Вавилово,71290,NaN,,2,54.811714,55.849997,Республика Башкортостан,"Республика Башкортостан, Уфимский район, дерев..."
28,False,Банк Уралсиб,Давлеканово,61779,NaN,,2,54.222727,55.031270,Республика Башкортостан,"Республика Башкортостан, Давлеканово, улица 50..."
30,False,Банк Уралсиб,Давлеканово,61698,NaN,,2,54.225354,55.033606,Республика Башкортостан,"Республика Башкортостан, Давлеканово, Карански..."
31,False,Банк Уралсиб,Давлеканово,61347,NaN,,2,54.213077,55.005354,Республика Башкортостан,"Республика Башкортостан, Давлеканово, улица Ал..."
33,False,Ак Барс,Камское Устье,71329,NaN,,2,55.201560,55.201560,Республика Башкортостан,"Республика Башкортостан, Кушнаренковский район"
39,False,Банк Уралсиб,Нефтекамск,61956,NaN,,2,56.106188,54.220379,Республика Башкортостан,"Республика Башкортостан, Нефтекамск, улица Дзе..."
58,False,Альфа-Банк,Октябрьский,81506,NaN,,2,58.082799,52.865862,Республика Башкортостан,"Удмуртская Республика, Глазовский район, село ..."


In [62]:
df_atm.rename(columns={'name': 'comment'}, inplace=True)

## Объединяем два датасета в один

In [63]:
from collections import Counter

In [76]:
df_comment = pd.concat([
    df_atm[['bank', 'address', 'comment', 'lat', 'lng', 'metro', 'access24h', 'id']],
    df_banki[['bank', 'address', 'comment', 'lat', 'lon', 'access24h']]\
        .rename(columns={'lon': 'lng'})
], sort=False)
df_comment.head()

,bank,address,comment,lat,lng,metro,access24h,id
0,Росбанк,"Красноярск, улица Партизана Железняка, 35А",Бизнес Центр Сириус ХАБ ПАО РОСБАНК,56.035552,92.930698,NaN,False,45351.0
1,Росбанк,"Республика Адыгея, Тахтамукайский район, посёл...","магазин ""Декатлон""",45.013084,38.929328,NaN,False,44392.0
2,Альфа-Банк,"Республика Адыгея, Тахтамукайский район, посёл...",ТЦ «Мега Адыгея»,45.012358,38.928279,NaN,False,81782.0
3,Альфа-Банк,"Республика Адыгея, Тахтамукайский район, аул Т...",,44.929061,38.980093,NaN,True,83137.0
4,Газпромбанк,"Республика Бурятия, Улан-Удэ, Геологическая ул...",САО «ВСК»,51.810559,107.607134,NaN,False,75759.0


In [77]:
df_comment.to_csv('base_banks_join.csv', index=False, sep=',')